In [1]:
import numpy as np
import cv2
from collections import Counter

In [2]:
image_path = 'sar_1_gray.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = image.astype(np.float32)

In [3]:
np.savetxt('image.txt', image, fmt="%d")

In [4]:
def haar_transform(img):
    h, w = img.shape
    temp = np.zeros_like(img, dtype=np.float32)

    # Горизонтальное преобразование
    for i in range(h):
        for j in range(0, w, 2):
            if j + 1 < w:
                temp[i, j // 2] = (img[i, j] + img[i, j + 1]) / 2
                temp[i, j // 2 + w // 2] = (img[i, j] - img[i, j + 1]) / 2

    # Вертикальное преобразование
    result = np.zeros_like(temp, dtype=np.float32)
    for j in range(w):
        for i in range(0, h, 2):
            if i + 1 < h:
                result[i // 2, j] = (temp[i, j] + temp[i + 1, j]) / 2
                result[i // 2 + h // 2, j] = (temp[i, j] - temp[i + 1, j]) / 2

    # Разделение на субполосы
    ll = result[:h // 2, :w // 2]
    hl = result[h // 2:, :w // 2]
    lh = result[:h // 2, w // 2:]
    hh = result[h // 2:, w // 2:]

    return ll, lh, hl, hh

In [5]:
ll, lh, hl, hh = haar_transform(image)
n_quants = 4

In [6]:
def quantize(coeffs, n_quants):
    min_val = coeffs.min()
    max_val = coeffs.max()
    step = (max_val - min_val) / n_quants

    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized, min_val, step


lh_q, lh_min, lh_step = quantize(lh, n_quants)
hl_q, hl_min, hl_step = quantize(hl, n_quants)
hh_q, hh_min, hh_step = quantize(hh, n_quants)

In [7]:
def run_length_encode(data):
    encoded = [(value, count) for value, count in Counter(data.flatten()).items()]
    return encoded


lh_rle = run_length_encode(lh_q)
hl_rle = run_length_encode(hl_q)
hh_rle = run_length_encode(hh_q)

In [8]:
with open('wavelet_data.txt', 'w') as f:
    # LL сохраняем полностью
    np.savetxt(f, ll, fmt='%d')
    f.write('\n')

    # LH
    for value, count in lh_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')

    # HL
    for value, count in hl_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')

    # HH
    for value, count in hh_rle:
        f.write(f"{value} {count}\n")

In [9]:
original_size = image.nbytes

with open('wavelet_data.txt', 'r') as f:
    compressed_size = len(f.read().encode('utf-8'))

print(f"Размер оригинального изображения: {original_size} байт")
print(f"Размер сжатого изображения: {compressed_size} байт")
print(f"Сжалось в: {original_size / compressed_size:.2f}")

Размер оригинального изображения: 960000 байт
Размер сжатого изображения: 190884 байт
Сжалось в: 5.03
